Development of B1: "the revised base model"

in this notebook, all input images and corresponding ground truths are brought to the same size 256 x256. after doing this, Up-sampling block needed to restore the image to different input image sizes won't be required. 
Also, regions of non-interest are treated as below - 
1. Label between 0 to 0.19 is assigned ‘0’
2. Label between 0.19 to 0.8 is assigned ‘-1’
3. Label between 0.8 to 1 is assigned ‘1’ 

In [5]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8 MB 26 kB/s 
     |████████████████████████████████| 3.8 MB 28.0 MB/s 
     |████████████████████████████████| 448 kB 71.6 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=0a1b326747d440c49e9d57d0541a9ae5fd81cbc243599474515c8c94cefd326e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [8]:
pip install keras==2.3.0

     |████████████████████████████████| 377 kB 4.3 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0


In [9]:
!pip install -U scikit-learn==0.24

     |████████████████████████████████| 22.3 MB 3.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


In [10]:
# importing the required package
import numpy as np
import tensorflow as tf
import random as rn
import os, sys
import time
from keras import backend as K
import PIL
from PIL import Image
import PIL.Image
import keras, glob
from keras.preprocessing import image as kImage
from skimage.transform import pyramid_gaussian
from sklearn.utils import compute_class_weight
#from FgSegNet_M_S_module import FgSegNet_M_S_module # Prashanna has to check for this
from keras.utils.data_utils import get_file
from keras.models import Model
from keras.layers import Activation, Input, Dropout, BatchNormalization, SpatialDropout2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras import regularizers
from keras.engine.topology import Layer
from keras.utils import conv_utils
from keras.legacy import interfaces
from keras.engine.base_layer import InputSpec

#from my_upsampling_2d import MyUpSampling2D # Prashanna to check the need for this

Using TensorFlow backend.


In [11]:
# print the entire array without truncation
np.set_printoptions(threshold=sys.maxsize)

In [12]:
# For reproducing the environment
np.random.seed(42)
rn.seed(12345)
tf.random.set_seed(1234)

In [13]:
# Define Data Generator (Inspired by FgSegNet)
def generateData(train_dir, dataset_dir, scene, method_name):
    void_label = -1. # defining label to be assigned to non-RoI
    
    # Given ground-truths, load training frames
    # ground-truths end with '*.png'
    # training frames end with '*.jpg'
    
    # given ground-truths, load inputs  
    Y_list = glob.glob(os.path.join(train_dir, '*.png')) #stores the location of all the 50 files in ground truth.
    X_list= glob.glob(os.path.join(dataset_dir, 'input','*.jpg')) #stores the location of all files in input directory

    #for every file name in Ground Truth (Y) find the corresponding file in Input directory (X)
    X_list_temp = []
    for i in range(len(Y_list)):
        Y_name = os.path.basename(Y_list[i]) 
        Y_name = Y_name.split('.')[0]
        Y_name = Y_name.split('gt')[1]
        for j in range(len(X_list)):
            X_name = os.path.basename(X_list[j])
            X_name = X_name.split('.')[0]
            X_name = X_name.split('in')[1]
            if (Y_name == X_name):
                X_list_temp.append(X_list[j])
                break
            
    X_list = X_list_temp # X corresponding to Ground Truth created
    
    # X must be corresponded to Y
    X_list = sorted(X_list)
    Y_list = sorted(Y_list)

    # load training data
    X = []
    Y = []
    for i in range(len(X_list)):
        x = kImage.load_img(X_list[i]) # load image
        x = x.resize((256, 256))
        x = kImage.img_to_array(x) # convert the loaded image to array
        X.append(x) # create X
        
        x = kImage.load_img(Y_list[i], grayscale = True) # load Y image
        x = x.resize((256, 256))
        x = kImage.img_to_array(x) # convert to array
        shape = x.shape # shape of Y (height x width x 1)
        x /= 255.0 # normalize to fit the range 0 to 1
        x = x.reshape(-1) # flatten to find out pixels corresponding to non-ROI
        idx = np.where(np.logical_and(x>0.19, x<0.8))[0] # find non-ROI
        if (len(idx)>0):
            x[idx] = void_label # assigning -1 to the non-ROI region of image
        x = x.reshape(shape) # reshape the image back to original size
        x = np.floor(x) #floor to the nearest integer. so now we have only 3 values: -1, 0, 1
        Y.append(x) #create list of Y
   
   # Convert X and Y to array     
    X = np.asarray(X)
    Y = np.asarray(Y)
        
    # We do not consider temporal data. soo shuffle 0 to 49 and then apply the shuffled list to X and Y
    idx = list(range(X.shape[0])) 
    np.random.shuffle(idx)
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    
    # compute class weights
    cls_weight_list = [] # initialize the list
    for i in range(Y.shape[0]): # for in in range 50
        y = Y[i].reshape(-1) # flatten Y
        idx = np.where(y!=void_label)[0] # find out the pixels that are marked as -1 (non ROI)
        if(len(idx)>0):
            y = y[idx] #find out all the non -1 entries in Y
        lb = np.unique(y) #  0., 1
        cls_weight = compute_class_weight('balanced', lb , y)
        class_0 = cls_weight[0]
        class_1 = cls_weight[1] if len(lb)>1 else 1.0
        
        cls_weight_dict = {0:class_0, 1: class_1}
        cls_weight_list.append(cls_weight_dict)

    cls_weight_list = np.asarray(cls_weight_list)
    
    return [X,Y,cls_weight_list]

In [14]:
def train(results, scene, mdl_path, vgg_weights_path, method_name, category):
    img_shape = results[0][0].shape # (height, width, channel)
        
    model = initModel_S(lr, reg, img_shape, scene, vgg_weights_path)
    
    chk = keras.callbacks.ModelCheckpoint(mdl_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    redu = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor, patience=num_patience, verbose=1, mode='auto')
    early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=0, mode='auto')

    t0=time.time()
    model.fit(results[0], results[1], validation_split=val_split, epochs=max_epochs, batch_size=batch_size, 
              callbacks=[redu, early], verbose=1, class_weight=results[2], shuffle = True)
    print ("training time:",category, ">", scene,  round(time.time()-t0, 3), "s")
    model.save(mdl_path)
        
    del model, results, chk, redu, early

In [15]:
def VGG16(x): 
  # Block 1
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_last')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
  # Block 2
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
  # Block 3
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    
  # Block 4
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
  x = Dropout(0.5, name='dr1')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
  x = Dropout(0.5, name='dr2')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
  x = Dropout(0.5, name='dr3')(x)
        
  return x

In [16]:
def transposedConv(x, reg):
  # block 5
  x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block5_tconv1', 
                                                kernel_regularizer=regularizers.l2(reg))(x)
  x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='block5_tconv2')(x)
  x = Conv2DTranspose(512, (1, 1), activation='relu', padding='same', name='block5_tconv3')(x)
        
  # block 6
  x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block6_tconv1', 
                                                kernel_regularizer=regularizers.l2(reg))(x)
  x = Conv2DTranspose(64, (5, 5), strides=(2, 2), activation='relu', padding='same', name='block6_tconv2')(x)
  x = Conv2DTranspose(256, (1, 1), activation='relu', padding='same', name='block6_tconv3')(x)
        
  # block 7
  x = Conv2DTranspose(64, (1, 1), activation='relu', padding='same', name='block7_tconv1', 
                                                kernel_regularizer=regularizers.l2(reg))(x)
  x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', name='block7_tconv2')(x)
  x = Conv2DTranspose(128, (1, 1), activation='relu', padding='same', name='block7_tconv3')(x)
        
  # block 8
  x = Conv2DTranspose(64, (5, 5), strides=(2, 2), activation='relu', padding='same', name='block8_conv1', 
                                                kernel_regularizer=regularizers.l2(reg))(x)
        
  # block 9
  x = Conv2DTranspose(1, (1, 1), padding='same', name='block9_conv1')(x)
  x = Activation('sigmoid')(x)
        
  return x

In [17]:
def acc(y_true, y_pred):
  void_label = -1.
  y_pred = tf.reshape(y_pred, [-1])
  y_true = tf.reshape(y_true, [-1])
  idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
  y_pred = tf.gather_nd(y_pred, idx) 
  y_true = tf.gather_nd(y_true, idx)
  return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)

In [18]:
def custom_f1(y_true, y_pred):    
    void_label = -1.
    y_pred = tf.reshape(y_pred, [-1])
    y_true = tf.reshape(y_true, [-1])
    idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
    y_pred = tf.gather_nd(y_pred, idx) 
    y_true = tf.gather_nd(y_true, idx)
    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
#Loss function
def loss(y_true, y_pred):
  void_label = -1.
  y_pred = K.reshape(y_pred, [-1])
  y_true = K.reshape(y_true, [-1])
  idx = tf.where(tf.not_equal(y_true, tf.constant(void_label, dtype=tf.float32)))
  y_pred = tf.gather_nd(y_pred, idx) 
  y_true = tf.gather_nd(y_true, idx)
  return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

In [20]:
def FPM(x):
  x1 = MaxPooling2D((2, 2), strides=(1,1), padding='same')(x)
  x1 = Conv2D(64, (1, 1), padding='same')(x1)
        
  x2 = Conv2D(64, (3, 3), padding='same')(x)
        
  x3 = Conv2D(64, (3, 3), padding='same', dilation_rate=4)(x)
        
  x4 = Conv2D(64, (3, 3), padding='same', dilation_rate=8)(x)
        
  x5 = Conv2D(64, (3, 3), padding='same', dilation_rate=16)(x)
        
  x = keras.layers.concatenate([x1, x2, x3, x4, x5], axis=-1)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = SpatialDropout2D(0.25)(x)
  return x

In [21]:
#class MyUpSampling2D(Layer):
    
#    #@interfaces.legacy_upsampling2d_support
#    def __init__(self, size=(2, 2), num_pixels = (0, 0), data_format='channels_last', method_name='FgSegNet_S', **kwargs):
#        super(MyUpSampling2D, self).__init__(**kwargs)
#        self.data_format = K.normalize_data_format(data_format)
#        self.size = conv_utils.normalize_tuple(size, 2, 'size')
#        self.input_spec = InputSpec(ndim=4)
#        self.num_pixels = num_pixels
#        self.method_name = method_name
#
#    def compute_output_shape(self, input_shape):
#        if self.data_format == 'channels_last':
#            height = self.size[0] * input_shape[1] + self.num_pixels[0] if input_shape[1] is not None else None
#            width = self.size[1] * input_shape[2] + self.num_pixels[1] if input_shape[2] is not None else None
#            return (input_shape[0], height, width, input_shape[3])
#        
#        else:
#            raise ValueError('Invalid data_format:', self.data_format)
#        
#    def call(self, inputs):
#        return resize_images(inputs, self.size[0], self.size[1], self.data_format, self.num_pixels, self.method_name)
#
#    def get_config(self):
#        config = {'size': self.size,
#                  'data_format': self.data_format,
#                  'num_pixels': self.num_pixels}
#        base_config = super(MyUpSampling2D, self).get_config()
#        return dict(list(base_config.items()) + list(config.items()))

In [22]:
def initModel_S(lr, reg, img_shape, scene, vgg_weights_path):
  assert len(img_shape)==3
  h, w, d = img_shape
        
  input_1 = Input(shape=(h, w, d), name='input')
  vgg_layer_output = VGG16(input_1)
  model = Model(inputs=input_1, outputs=vgg_layer_output, name='model')
  #model.summary()
  model.load_weights(vgg_weights_path, by_name=True)
        
  unfreeze_layers = ['block4_conv1','block4_conv2', 'block4_conv3']
  for layer in model.layers:
    if(layer.name not in unfreeze_layers):
      layer.trainable = False
                
  x = model.output
        
  #x1_ups = {'streetCornerAtNight':(0,1), 'tramStation':(1,0), 'turbulence2':(1,0)}
  #for key, val in x1_ups.items():
  #  if scene==key:
      # upscale by adding number of pixels to each dim.
  #    x = MyUpSampling2D(size=(1,1), num_pixels=val)(x)
  #    break
                
  x = FPM(x)
  x = transposedConv(x, reg)
        
      
  vision_model = Model(inputs=input_1, outputs=x, name='vision_model')
  opt = keras.optimizers.RMSprop(lr = lr, rho=0.9, epsilon=1e-08, decay=0.)
        
  c_loss = loss
  c_acc = acc
  c_custom_f1 = custom_f1
            
  vision_model.compile(loss=c_loss, optimizer=opt, metrics=[c_custom_f1, c_acc])
  vision_model.summary()
  return vision_model

In [23]:
# Main Function
# dataset for ablative experiment
dataset = {
            'baseline':['highway'],
            'cameraJitter':['badminton'],
            'badWeather':['skating'],
            'dynamicBackground':['boats'],
            'intermittentObjectMotion':['abandonedBox'],
            'lowFramerate':['port_0_17fps'],
            'nightVideos':['bridgeEntry'],
            'PTZ':['continuousPan'],
            'shadow':['backdoor'],
            'thermal':['corridor'],
            'turbulence':['turbulence0']
}

In [24]:
method_name = 'ModelB1' 

num_frames = 50 # default 50 frames
reduce_factor = 0.1
num_patience = 6
lr = 1e-4
reg=5e-4
max_epochs = 50
val_split = 0.2 #Assumed. For num_frames = 50; 40 frames will be used for training and 10 frames will be used for validation
batch_size = 1 #Assumed to be 1

In [25]:
main_dir = os.path.join('/content/drive/MyDrive/FgSegNet', method_name)
main_mdl_dir = os.path.join(main_dir, 'CDnet', 'models' + str(num_frames))
vgg_weights_path = '/content/drive/MyDrive/FgSegNet/FgSegNet/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [26]:
for category, scene_list in dataset.items():
    
    mdl_dir = os.path.join(main_mdl_dir, category)
    if not os.path.exists(mdl_dir):
        os.makedirs(mdl_dir)
        
    for scene in scene_list:
        print ('Training ->>> ' + category + ' / ' + scene)
        
        # training frame path and dataset2014 path
        train_dir = os.path.join('/content/drive/MyDrive/FgSegNet', 'FgSegNet_dataset2014', category, scene + str(num_frames))
        dataset_dir = os.path.join('/content/drive/MyDrive/FgSegNet', 'CDnet2014_dataset', category, scene)
        results = generateData(train_dir, dataset_dir, scene, method_name)
        
        mdl_path = os.path.join(mdl_dir, 'mdl_' + scene + '.h5')
        train(results, scene, mdl_path, vgg_weights_path, method_name, category)
        del results

Training ->>> baseline / highway
Model: "vision_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input[0][0]                      
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
______________________________________________________